# Initialisation

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import json

# Create instances with all stations

In [2]:
daily_trains_demand_pre_covid = pd.read_csv('../data/curated/train_demand/daily_trains_demand_pre_covid.csv')
daily_trains_demand_post_covid = pd.read_csv('../data/curated/train_demand/daily_trains_demand_post_covid.csv')
mean_daily_trains_demand_post_covid = pd.read_csv('../data/curated/train_demand/mean_daily_trains_demand_post_covid.csv')

In [15]:
daily_trains_demand_post_covid_weekday = daily_trains_demand_post_covid[(daily_trains_demand_post_covid['Weekday'] == 1)]
daily_trains_demand_post_covid_weekend = daily_trains_demand_post_covid[(daily_trains_demand_post_covid['Weekday'] == 0)]

In [34]:
# add rows together
mean_daily_trains_demand_post_covid_weekday = pd.DataFrame()

for id, station_data in daily_trains_demand_post_covid_weekday.drop(['Business_Date', 'Weekday', 'PublicHoliday', 'Unnamed: 0'], axis=1).groupby('Station_Name'):
    station_data = station_data.drop('Station_Name', axis=1)
    station_data_mean = station_data.mean()
    station_data_mean = pd.DataFrame(station_data_mean).T
    station_data_mean['Station_Name'] = id
    mean_daily_trains_demand_post_covid_weekday = pd.concat([mean_daily_trains_demand_post_covid_weekday, station_data_mean], axis=0)


mean_daily_trains_demand_post_covid_weekend = pd.DataFrame()

for id, station_data in daily_trains_demand_post_covid_weekend.drop(['Business_Date', 'Weekday', 'PublicHoliday', 'Unnamed: 0'], axis=1).groupby('Station_Name'):
    station_data = station_data.drop('Station_Name', axis=1)
    station_data_mean = station_data.mean()
    station_data_mean = pd.DataFrame(station_data_mean).T
    station_data_mean['Station_Name'] = id
    mean_daily_trains_demand_post_covid_weekend = pd.concat([mean_daily_trains_demand_post_covid_weekend, station_data_mean], axis=0)

In [37]:
mean_daily_trains_demand_post_covid_weekday['log_Total_Demand'] = np.log(mean_daily_trains_demand_post_covid_weekday['Total_Demand'])
mean_daily_trains_demand_post_covid_weekend['log_Total_Demand'] = np.log(mean_daily_trains_demand_post_covid_weekend['Total_Demand'])
mean_daily_trains_demand_post_covid_weekday['log_Passenger_Alightings'] = np.log(mean_daily_trains_demand_post_covid_weekday['Passenger_Alightings'])
mean_daily_trains_demand_post_covid_weekend['log_Passenger_Alightings'] = np.log(mean_daily_trains_demand_post_covid_weekend['Passenger_Alightings'])
mean_daily_trains_demand_post_covid_weekday['log_Passenger_Boardings'] = np.log(mean_daily_trains_demand_post_covid_weekday['Passenger_Boardings'])
mean_daily_trains_demand_post_covid_weekend['log_Passenger_Boardings'] = np.log(mean_daily_trains_demand_post_covid_weekend['Passenger_Boardings'])

In [38]:
def get_daily_demand_in_rows(demand_df: pd.DataFrame, feature: str):

    try:
        df_with_feature = demand_df[['Station_Name', 'Business_Date', feature]]
    except:
        df_with_feature = demand_df[['Station_Name', feature]]

    station_df_list = []

    for id, station_df in tqdm(df_with_feature.groupby('Station_Name')):

        station_df = station_df.rename({feature: f'{feature}_{id}'}, axis=1)
        station_df = station_df.drop('Station_Name', axis=1)
        station_df_list.append(station_df)

    for i, station_df in enumerate(station_df_list):
        if i == 0:
            merged_df = station_df
        else:
            try:
                merged_df = pd.merge(merged_df, station_df, on='Business_Date', how='outer')
            except:
                merged_df = pd.concat([merged_df, station_df], axis=0)

    merged_df = merged_df.fillna(0)
    return merged_df

In [39]:
log_demand_precovid = get_daily_demand_in_rows(daily_trains_demand_pre_covid, 'log_Total_Demand')
log_demand_postcovid = get_daily_demand_in_rows(daily_trains_demand_post_covid, 'log_Total_Demand')
log_alighting_precovid = get_daily_demand_in_rows(daily_trains_demand_pre_covid, 'log_Passenger_Alightings')
log_alighting_postcovid = get_daily_demand_in_rows(daily_trains_demand_post_covid, 'log_Passenger_Alightings')
log_boarding_precovid = get_daily_demand_in_rows(daily_trains_demand_pre_covid, 'log_Passenger_Boardings')
log_boarding_postcovid = get_daily_demand_in_rows(daily_trains_demand_post_covid, 'log_Passenger_Boardings')

100%|██████████| 223/223 [00:00<00:00, 2153.65it/s]


In [41]:
mean_daily_trains_demand_post_covid_weekday = mean_daily_trains_demand_post_covid_weekday.sort_values('Station_Name')
mean_daily_trains_demand_post_covid_weekday = mean_daily_trains_demand_post_covid_weekday.set_index('Station_Name')

mean_daily_trains_demand_post_covid_weekend = mean_daily_trains_demand_post_covid_weekend.sort_values('Station_Name')
mean_daily_trains_demand_post_covid_weekend = mean_daily_trains_demand_post_covid_weekend.set_index('Station_Name')

In [43]:
def get_daily_demand_in_rows_inference(df, feature):
    return df[[feature]].T

In [44]:
log_mean_demand_postcovid_weekday = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekday, 'log_Total_Demand')
log_mean_alighting_postcovid_weekday = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekday, 'log_Passenger_Alightings')
log_mean_boarding_postcovid_weekday = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekday, 'log_Passenger_Boardings')

log_mean_demand_postcovid_weekend = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekend, 'log_Total_Demand')
log_mean_alighting_postcovid_weekend = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekend, 'log_Passenger_Alightings')
log_mean_boarding_postcovid_weekend = get_daily_demand_in_rows_inference(mean_daily_trains_demand_post_covid_weekend, 'log_Passenger_Boardings')

In [45]:
os.makedirs('../data/curated/ML_features', exist_ok=True)

In [46]:
log_demand_precovid.to_csv('../data/curated/ML_features/log_demand_precovid.csv', index=False)
log_demand_postcovid.to_csv('../data/curated/ML_features/log_demand_postcovid.csv', index=False)
log_alighting_precovid.to_csv('../data/curated/ML_features/log_alighting_precovid.csv', index=False)
log_alighting_postcovid.to_csv('../data/curated/ML_features/log_alighting_postcovid.csv', index=False)
log_boarding_precovid.to_csv('../data/curated/ML_features/log_boarding_precovid.csv', index=False)
log_boarding_postcovid.to_csv('../data/curated/ML_features/log_boarding_postcovid.csv', index=False)

log_mean_demand_postcovid_weekday.to_csv('../data/curated/ML_features/log_mean_demand_postcovid_weekday.csv', index=False)
log_mean_alighting_postcovid_weekday.to_csv('../data/curated/ML_features/log_mean_alighting_postcovid_weekday.csv', index=False)
log_mean_boarding_postcovid_weekday.to_csv('../data/curated/ML_features/log_mean_boarding_postcovid_weekday.csv', index=False)
log_mean_alighting_postcovid_weekend.to_csv('../data/curated/ML_features/log_mean_alighting_postcovid_weekend.csv', index=False)
log_mean_boarding_postcovid_weekend.to_csv('../data/curated/ML_features/log_mean_boarding_postcovid_weekend.csv', index=False)
log_mean_demand_postcovid_weekend.to_csv('../data/curated/ML_features/log_mean_demand_postcovid_weekend.csv', index=False)

# Weight demand data

In [47]:
station_weights_withSA2 = pd.read_csv('../data/curated/ML_features/station_weights_withSA2.csv')
station_weights = pd.read_csv('../data/curated/ML_features/station_weights.csv')

station_with_sa2_list_dict = {k:i for i, k in enumerate(station_weights_withSA2['Unnamed: 0'])}
station_list_dict = {k:i for i, k in enumerate(station_weights['Unnamed: 0'])}
reverse_station_with_sa2_list_dict = {i:k for i, k in enumerate(station_weights_withSA2['Unnamed: 0'])}
reverse_station_list_dict = {i:k for i, k in enumerate(station_weights['Unnamed: 0'])}

station_weights_withSA2.set_index('Unnamed: 0', inplace=True)
station_weights.set_index('Unnamed: 0', inplace=True)

In [48]:
with open('../data/curated/ML_features/station_weights_withSA2.json', 'w') as f:
    json.dump(station_with_sa2_list_dict, f)

with open('../data/curated/ML_features/station_weights.json', 'w') as f:
    json.dump(station_list_dict, f)

In [49]:
log_demand_postcovid.set_index('Business_Date', inplace=True)
log_alighting_postcovid.set_index('Business_Date', inplace=True)
log_boarding_postcovid.set_index('Business_Date', inplace=True)

In [50]:
def get_weighted_demand(log_demand: pd.DataFrame, station_weights: pd.DataFrame, reverse_station_list_dict: dict):
    results = []

    for station, row in tqdm(station_weights.iterrows()):
        weights = row.values

        for date, row in log_demand.iterrows():
            demand = row.values

            weighted_demand = weights * demand

            # Append to results list
            results.append({
                'Station': station,
                'Date': date,
                **{f'{reverse_station_list_dict[i]}': wd for i, wd in enumerate(weighted_demand)}
            })

    # Convert results list to DataFrame
    weighted_demand_df = pd.DataFrame(results)

    return weighted_demand_df

In [51]:
weighted_log_demand_postcovid = get_weighted_demand(log_demand_postcovid, station_weights, reverse_station_list_dict)
weighted_log_alighting_postcovid = get_weighted_demand(log_alighting_postcovid, station_weights, reverse_station_list_dict)
weighted_log_boarding_postcovid = get_weighted_demand(log_boarding_postcovid, station_weights, reverse_station_list_dict)

223it [00:09, 23.14it/s]
223it [00:09, 22.54it/s]
223it [00:09, 23.53it/s]


In [52]:
def get_weighted_demand_inference(log_demand: pd.DataFrame, station_weights: pd.DataFrame, reverse_station_list_dict: dict):
    results = []

    i = 0
    for station, row in tqdm(station_weights.iterrows()):
        weights = row.values[:223]
        if '(SA2)' not in station:
            continue
        else:
            station = station.split('(SA2)')[1]
 
        for date, row in log_demand.iterrows():
            demand = row.values

            weighted_demand = weights * demand

            # Append to results list
            results.append({
                'Station': reverse_station_list_dict[i+223],
                'Date': date,
                **{f'{reverse_station_list_dict[i]}': wd for i, wd in enumerate(weighted_demand)}
            })

            i += 1

    # Convert results list to DataFrame
    weighted_demand_df = pd.DataFrame(results)

    return weighted_demand_df

In [53]:
weighted_log_mean_demand_postcovid_weekday = get_weighted_demand_inference(log_mean_demand_postcovid_weekday, station_weights_withSA2, reverse_station_with_sa2_list_dict)
weighted_log_mean_alighting_postcovid_weekday = get_weighted_demand_inference(log_mean_alighting_postcovid_weekday, station_weights_withSA2, reverse_station_with_sa2_list_dict)
weighted_log_mean_boarding_postcovid_weekday = get_weighted_demand_inference(log_mean_boarding_postcovid_weekday, station_weights_withSA2, reverse_station_with_sa2_list_dict)

weighted_log_mean_demand_postcovid_weekend = get_weighted_demand_inference(log_mean_demand_postcovid_weekend, station_weights_withSA2, reverse_station_with_sa2_list_dict)
weighted_log_mean_alighting_postcovid_weekend = get_weighted_demand_inference(log_mean_alighting_postcovid_weekend, station_weights_withSA2, reverse_station_with_sa2_list_dict)
weighted_log_mean_boarding_postcovid_weekend = get_weighted_demand_inference(log_mean_boarding_postcovid_weekend, station_weights_withSA2, reverse_station_with_sa2_list_dict)

442it [00:00, 15571.12it/s]


442it [00:00, 15796.41it/s]
442it [00:00, 13916.78it/s]
442it [00:00, 16096.08it/s]
442it [00:00, 15464.36it/s]
442it [00:00, 16163.16it/s]


In [54]:
weighted_log_mean_demand_postcovid_weekday.drop('Date', axis=1, inplace=True)
weighted_log_mean_alighting_postcovid_weekday.drop('Date', axis=1, inplace=True)
weighted_log_mean_boarding_postcovid_weekday.drop('Date', axis=1, inplace=True)

weighted_log_mean_demand_postcovid_weekend.drop('Date', axis=1, inplace=True)
weighted_log_mean_alighting_postcovid_weekend.drop('Date', axis=1, inplace=True)
weighted_log_mean_boarding_postcovid_weekend.drop('Date', axis=1, inplace=True)

In [55]:
weighted_log_demand_postcovid.to_parquet('../data/curated/ML_features/weighted_log_demand_postcovid.parquet', index=False)
weighted_log_alighting_postcovid.to_parquet('../data/curated/ML_features/weighted_log_alighting_postcovid.parquet', index=False)
weighted_log_boarding_postcovid.to_parquet('../data/curated/ML_features/weighted_log_boarding_postcovid.parquet', index=False)

weighted_log_mean_demand_postcovid_weekday.to_parquet('../data/curated/ML_features/weighted_log_mean_demand_postcovid_weekday.parquet', index=False)
weighted_log_mean_alighting_postcovid_weekday.to_parquet('../data/curated/ML_features/weighted_log_mean_alighting_postcovid_weekday.parquet', index=False)
weighted_log_mean_boarding_postcovid_weekday.to_parquet('../data/curated/ML_features/weighted_log_mean_boarding_postcovid_weekday.parquet', index=False)

weighted_log_mean_demand_postcovid_weekend.to_parquet('../data/curated/ML_features/weighted_log_mean_demand_postcovid_weekend.parquet', index=False)
weighted_log_mean_alighting_postcovid_weekend.to_parquet('../data/curated/ML_features/weighted_log_mean_alighting_postcovid_weekend.parquet', index=False)
weighted_log_mean_boarding_postcovid_weekend.to_parquet('../data/curated/ML_features/weighted_log_mean_boarding_postcovid_weekend.parquet', index=False)

# Join up data

In [56]:
daily_trains_demand_post_covid.drop(['Unnamed: 0', 'Passenger_Boardings', 'Passenger_Alightings', 'Total_Demand', 'log_Passenger_Boardings', 'log_Passenger_Alightings'], axis = 1, inplace=True)

In [57]:
rainfall_df = pd.read_csv('../data/curated/ML_features/rainfall_Station_SA2.csv')
rainfall_df_stations = rainfall_df[~rainfall_df['Station_Na'].isna()][['mean_rainfall_value', 'Station_Na']]
rainfall_df_sa2 = rainfall_df[rainfall_df['Station_Na'].isna()][['mean_rainfall_value', 'SA2_NAME21']]
rainfall_df_sa2.rename({'SA2_NAME21': 'Station'}, axis=1, inplace=True)

census_and_buildings_postcovid = pd.read_csv('../data/curated/ML_features/census_and_buildings_postcovid.csv')

In [58]:
station_ML_data = daily_trains_demand_post_covid.merge(rainfall_df_stations, left_on='Station_Name', right_on='Station_Na', how='left')
station_ML_data.drop(['Station_Na'], axis=1, inplace=True)
station_ML_data = station_ML_data.merge(census_and_buildings_postcovid[census_and_buildings_postcovid['point_type'] == 'station'], left_on = 'Station_Name', right_on='Point Name', how = 'left')

station_ML_data = station_ML_data.merge(weighted_log_demand_postcovid, left_on=['Business_Date', 'Station_Name'], right_on=['Date', 'Station'], how='left', suffixes=('', '_demand'))
station_ML_data = station_ML_data.merge(weighted_log_alighting_postcovid, left_on=['Business_Date', 'Station_Name'], right_on=['Date', 'Station'], how='left', suffixes=('', '_alighting'))
station_ML_data = station_ML_data.merge(weighted_log_boarding_postcovid, left_on=['Business_Date', 'Station_Name'], right_on=['Date', 'Station'], how='left', suffixes=('', '_boarding'))
station_ML_data.drop(['Point Name', 'point_type', 'Station_alighting', 'Station_boarding', 'Date_alighting', 'Date_boarding', 'Station'], inplace=True, axis = 1)

In [59]:
weighted_log_mean_demand_postcovid_weekday['Station'] = weighted_log_mean_demand_postcovid_weekday['Station'].apply(lambda x: x.split('(SA2)')[1])
weighted_log_mean_alighting_postcovid_weekday['Station'] = weighted_log_mean_alighting_postcovid_weekday['Station'].apply(lambda x: x.split('(SA2)')[1])
weighted_log_mean_boarding_postcovid_weekday['Station'] = weighted_log_mean_boarding_postcovid_weekday['Station'].apply(lambda x: x.split('(SA2)')[1])

weighted_log_mean_demand_postcovid_weekend['Station'] = weighted_log_mean_demand_postcovid_weekend['Station'].apply(lambda x: x.split('(SA2)')[1])
weighted_log_mean_alighting_postcovid_weekend['Station'] = weighted_log_mean_alighting_postcovid_weekend['Station'].apply(lambda x: x.split('(SA2)')[1])
weighted_log_mean_boarding_postcovid_weekend['Station'] = weighted_log_mean_boarding_postcovid_weekend['Station'].apply(lambda x: x.split('(SA2)')[1])

In [60]:
SA2_ML_data_weekday = rainfall_df_sa2.copy()
SA2_ML_data_weekday = SA2_ML_data_weekday.merge(census_and_buildings_postcovid[census_and_buildings_postcovid['point_type'] == 'suburb'], left_on = 'Station', right_on='Point Name', how = 'left')

SA2_ML_data_weekday = SA2_ML_data_weekday.merge(weighted_log_mean_demand_postcovid_weekday, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_demand'))
SA2_ML_data_weekday = SA2_ML_data_weekday.merge(weighted_log_mean_alighting_postcovid_weekday, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_alighting'))
SA2_ML_data_weekday = SA2_ML_data_weekday.merge(weighted_log_mean_boarding_postcovid_weekday, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_boarding'))

SA2_ML_data_weekday.drop(['Point Name', 'point_type'], axis=1, inplace=True)
SA2_ML_data_weekday.rename({'Station': 'Station Name'}, axis=1, inplace=True)

SA2_ML_data_weekend = rainfall_df_sa2.copy()
SA2_ML_data_weekend = SA2_ML_data_weekend.merge(census_and_buildings_postcovid[census_and_buildings_postcovid['point_type'] == 'suburb'], left_on = 'Station', right_on='Point Name', how = 'left')

SA2_ML_data_weekend = SA2_ML_data_weekend.merge(weighted_log_mean_demand_postcovid_weekend, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_demand'))
SA2_ML_data_weekend = SA2_ML_data_weekend.merge(weighted_log_mean_alighting_postcovid_weekend, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_alighting'))
SA2_ML_data_weekend = SA2_ML_data_weekend.merge(weighted_log_mean_boarding_postcovid_weekend, left_on=['Station'], right_on=['Station'], how='left', suffixes = ('', '_boarding'))

SA2_ML_data_weekend.drop(['Point Name', 'point_type'], axis=1, inplace=True)
SA2_ML_data_weekend.rename({'Station': 'Station Name'}, axis=1, inplace=True)

In [84]:
SA2_ML_data_weekday['Weekday'] = 1
SA2_ML_data_weekend['Weekday'] = 0

SA2_ML_data = pd.concat([SA2_ML_data_weekday, SA2_ML_data_weekend], axis=0)

In [61]:
os.makedirs('../data/curated/ML_data', exist_ok=True)

In [85]:
station_ML_data.to_parquet('../data/curated/ML_data/station_ML_data.parquet', index=False)
SA2_ML_data.to_parquet('../data/curated/ML_data/SA2_ML_data.parquet', index=False)

# Train Test Split and Normalisation

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [72]:
business_dates = station_ML_data['Business_Date'].unique()

train_dates, val_test_dates = train_test_split(business_dates, test_size=0.3, shuffle = False)
val_dates, test_dates = train_test_split(val_test_dates, test_size=0.5, shuffle = False)

ML_train_data = station_ML_data[station_ML_data['Business_Date'].isin(train_dates)]
ML_val_data = station_ML_data[station_ML_data['Business_Date'].isin(val_dates)]
ML_test_data = station_ML_data[station_ML_data['Business_Date'].isin(test_dates)]

In [83]:
ML_train_data.to_parquet('../data/curated/ML_data/ML_train_data.parquet', index=False)
ML_val_data.to_parquet('../data/curated/ML_data/ML_val_data.parquet', index=False)
ML_test_data.to_parquet('../data/curated/ML_data/ML_test_data.parquet', index=False)